# This file parse CUSIP from files downloaded from SEC

`links`: https://www.sec.gov/data/foiadocsfailsdatahtm

In [1]:
import os

import numpy as np
import pandas as pd

# Get CUSIP from SEC files

In [2]:
file_folder = 'CUSIP/'

text_files = [i for i in os.listdir(file_folder) if 'txt' in i]

text_files = [pd.read_csv(file_folder + name, delimiter='|', encoding = "ISO-8859-1") for name in text_files]

print('{} of text file loaded'.format(len(text_files)))


df = pd.concat(text_files)

df = df[['CUSIP', 'SYMBOL']]
df = df.groupby(['CUSIP', 'SYMBOL']).count().reset_index(drop=False) # since there is no other column count return nonthing

df.columns = ['CUSIP', 'Ticker']

print('dataframe row count: {}'.format(df.shape[0]))
print('unique CUSIP is collected: {}'.format(len(df.CUSIP.unique())))
print('unique ticker is collected: {}'.format(len(df.Ticker.unique())))

24 of text file loaded
dataframe row count: 23125
unique CUSIP is collected: 21659
unique ticker is collected: 20802


# Get CUSIP from Previously parsed files

In [3]:
df_1 = pd.read_csv('../0. Archive/cusip_ticker.csv')

print('dataframe row count: {}'.format(df_1.shape[0]))
print('unique CUSIP is collected: {}'.format(len(df_1.CUSIP.unique())))
print('unique ticker is collected: {}'.format(len(df_1.Ticker.unique())))

dataframe row count: 990
unique CUSIP is collected: 988
unique ticker is collected: 964


# Input in some CUSIP where I cannot automaticlly find corresponding tickers

In [4]:
CUSIP = {'056752908': 'BIDU', '86732YAC3': 'SUNE', 
         '88338TAB0': 'TBPH', '165167BZ9': 'CHK', 
         '165167CA3': 'CHK', '212894117': 'CPAA', 
         '60505104': 'BAC', '60505146': 'BAC', 
         '84670108': 'BRK.A'}


df_2 = pd.DataFrame({'CUSIP': list(CUSIP.keys()), 'Ticker': list(CUSIP.values())})

print('dataframe row count: {}'.format(df_2.shape[0]))
print('unique CUSIP is collected: {}'.format(len(df_2.CUSIP.unique())))
print('unique ticker is collected: {}'.format(len(df_2.Ticker.unique())))

dataframe row count: 9
unique CUSIP is collected: 9
unique ticker is collected: 7


### Merge the files together

In [5]:
df = pd.concat([df, df_1, df_2])

df = df.groupby(['CUSIP', 'Ticker']).count().reset_index(drop=False)

print('dataframe row count: {}'.format(df.shape[0]))
print('unique CUSIP is collected: {}'.format(len(df.CUSIP.unique())))
print('unique ticker is collected: {}'.format(len(df.Ticker.unique())))

dataframe row count: 23158
unique CUSIP is collected: 21676
unique ticker is collected: 20808


### Clean the data frame

In [6]:
def sort_by_len(df, col):
    
    ticker_index = df[col].apply(len).sort_values(ascending=False).index[0]
    return(df.loc[ticker_index, [col]])

In [7]:
df.Ticker = df.Ticker.replace({'BRKB': 'BRK.B', 'BRKA': 'BRK.A'}) # standardized BRK.B and BRK.A

df = df.groupby(['CUSIP']).apply(lambda x: sort_by_len(x, 'Ticker')).reset_index(drop=False) # get unique ticker by CUSIP

print('dataframe row count: {}'.format(df.shape[0]))
print('unique CUSIP is collected: {}'.format(len(df.CUSIP.unique())))
print('unique ticker is collected: {}'.format(len(df.Ticker.unique())))

dataframe row count: 21676
unique CUSIP is collected: 21676
unique ticker is collected: 19840


In [8]:
df.to_csv(file_folder + 'CUSIP.csv', index=False)